In [211]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sortedcontainers import SortedList
import math


Tách thành tập train và test


In [212]:
df = pd.read_csv('../dataset/rating_final.csv')
df = shuffle(df)

cut_off_length = int(0.8 * len(df))
df_train = df[:cut_off_length]
df_test = df[cut_off_length:]


In [213]:
user_to_movie = {}
movie_to_user = {}
movie_user_rating = {}

for row in df_train.values:
    movie_id, user_id, rate = row
    if movie_id not in user_to_movie:
        user_to_movie[user_id] = [movie_id]
    else:
        user_to_movie[user_id].append(movie_id)

    if movie_id not in movie_to_user:
        movie_to_user[movie_id] = [user_id]
    else:
        movie_to_user[movie_id].append(user_id)

    movie_user_rating[(movie_id, user_id)] = rate

In [214]:
    movie_user_rating_test = {}
    for row in df_test.values:
        movie_id, user_id, rate = row

        movie_user_rating_test[(movie_id, user_id)] = rate


In [215]:
number_of_movie = int(len(movie_to_user.keys()))
number_of_user = int(len(user_to_movie.keys()))

print(f'Number of movie: {number_of_movie}')
print(f'Number of user: {number_of_user}')


Number of movie: 490
Number of user: 1001


In [216]:
k_neighbor = 20
limit_neighbor = 5
neighbors = {}
averages = {}
deviations = {}

for movie_id_i in movie_to_user.keys():
    # get all user who rated for movie_id_i
    user_rated_movie_i = movie_to_user[movie_id_i]

    # get all rating of users gave for movie_id_i
    rating_user_for_movie_i = [
        movie_user_rating[(movie_id_i, user_id)] for user_id in user_rated_movie_i]

    # calculate avarage rating of all user gave for movie_id_i
    avg_rating_movie_i = np.mean(rating_user_for_movie_i)

    averages[movie_id] = avg_rating_movie_i
    
    sorted_list = SortedList()
    for movie_id_j in movie_to_user.keys():
        if movie_id_j == movie_id_i:
            continue

        # get all user who rated for movie_id_j
        user_rated_movie_j = movie_to_user[movie_id_j]

        common_users = list(
            set(user_rated_movie_i).intersection(set(user_rated_movie_j)))

        if len(common_users) < limit_neighbor:
            continue

        # get all rating of users gave for movie_id_j
        rating_user_for_movie_j = [
            movie_user_rating[(movie_id_j, user_id)] for user_id in user_rated_movie_j]

        # calculate avarage rating of all user gave for movie_id_j
        avg_rating_movie_j = np.mean(rating_user_for_movie_j)
        
        
        
        deviation = 0
        sigmoid_i = 0
        sigmoid_j = 0
        for user_id in common_users:
            dev_i = (
                movie_user_rating[(movie_id_i, user_id)] - avg_rating_movie_i)
            dev_j = (
                movie_user_rating[(movie_id_j, user_id)] - avg_rating_movie_j)
            deviation += dev_i * dev_j
            sigmoid_i += dev_i * dev_i
            sigmoid_j += dev_j * dev_j

        s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))
        sorted_list.add((s_ij, movie_id_j))
        if len(sorted_list) > k_neighbor:
            del sorted_list[0]

    neighbors[movie_id_i] = sorted_list
neighbors

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10984\1976594262.py:55: RuntimeWarning: invalid value encountered in double_scalars
  s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))


{'tt10366206': SortedList([(0.6328851010744728, 'tt23219684'), (0.6413536202363939, 'tt5290382'), (0.6623901123597219, 'tt3398228'), (0.6641492014607722, 'tt2571774'), (0.6702768940477772, 'tt0469494'), (0.6859186376686904, 'tt1190634'), (0.6917201899554855, 'tt1288558'), (0.7121451624373626, 'tt10048342'), (0.7147301428169088, 'tt0056592'), (0.7306890683443423, 'tt0361748'), (0.7437369878679306, 'tt3011894'), (0.7448903658938248, 'tt0804503'), (0.7581645143296557, 'tt0035446'), (0.7654133009811417, 'tt1628033'), (0.8693598988476071, 'tt4786824'), (0.8795270685150989, 'tt7137906'), (0.9607719009018588, 'tt0434706'), (nan, 'tt1675434'), (0.7832858539115344, 'tt0032551'), (0.8914915332651276, 'tt0278238')]),
 'tt0995832': SortedList([(-0.35634918407409405, 'tt4786824'), (-0.274940724287536, 'tt1641384'), (-0.20625208533820533, 'tt11813216'), (-0.20080360411400408, 'tt10640346'), (-0.12101141930168426, 'tt2582802'), (-0.06387246394314584, 'tt2707408'), (-0.030203024493289887, 'tt4093826')

In [217]:
def predict(movie_id_i, user_id):
    if movie_id_i not in movie_to_user or user_id not in user_to_movie:
        return None

    # get all user who rated for movie_id_i
    user_rated_movie_i = movie_to_user[movie_id_i]

    # get all rating of users gave for movie_id_i
    rating_user_for_movie_i = [movie_user_rating[(movie_id_i, uid)] for uid in user_rated_movie_i]

    # calculate average rating of all users who rated movie_id_i
    avg_rating_movie_i = np.mean(rating_user_for_movie_i)

    # calculate the predicted rating for movie_id_i by user_id
    predicted_rating = avg_rating_movie_i
    # check if the movie has neighbors
    if movie_id_i in neighbors:
        similarity_sum = 0
        weighted_rating_sum = 0

        # iterate over the neighbors of movie_id_i
        for similarity, neighbor_movie_id in neighbors[movie_id_i]:
            # check if the neighbor movie has been rated by any user
            if neighbor_movie_id in user_to_movie:
                # check if the user has rated the neighbor movie
                if user_id in user_to_movie[neighbor_movie_id]:
                    neighbor_rating = movie_user_rating[(neighbor_movie_id, user_id)]
                    avg_rating_neighbor = averages[neighbor_movie_id]
                    deviation = neighbor_rating - avg_rating_neighbor
                    weighted_rating_sum += similarity * deviation
                    similarity_sum += abs(similarity)
        if similarity_sum != 0:
            # calculate the final predicted rating
            predicted_rating += weighted_rating_sum / similarity_sum

    return predicted_rating




In [218]:
train_predictions = []
train_targets = []
# predict ratings for movies in df_test
for (m, u) in movie_user_rating.keys():
    prediction = predict(m, u)
    print(prediction)
    # save the prediction and target
    train_predictions.append(prediction)
    train_targets.append(target)

8.201069518716578
8.857142857142858
8.143877551020408
8.346206896551724
8.257707509881422
6.303225806451613
8.675739644970413
8.351832460732984
8.18734693877551
7.857758620689655
8.201069518716578
7.897560975609756
8.865
8.675739644970413
7.821153846153847
8.038983050847458
7.784444444444445
8.306666666666667
7.109767441860467
6.367647058823529
7.696385542168674
7.028037383177568
8.398863636363636
7.632894736842106
8.286170212765956
8.591111111111115
9.286486486486487
8.232394366197184
8.349655172413794
6.620869565217391
8.383425414364641
8.170754716981133
8.41764705882353
7.61263157894737
7.817088607594936
8.3003300330033
8.46880733944954
7.9239999999999995
8.688888888888888
8.488976377952756
9.169811320754716
8.440425531914894
8.16
8.16
8.52312925170068
6.280681818181818
8.488235294117647
8.257707509881422
7.764052287581698
9.249029126213593
7.61263157894737
8.27098445595855
6.95689655172414
8.16
8.293333333333335
8.51559633027523
8.16
8.488976377952756
7.076785714285713
8.2119402985

In [219]:
test_predictions = []
test_targets = []
for (m, u), target in movie_user_rating_test.items():
    # calculate the prediction for this movie
    prediction = predict(m, u)
    print(prediction)
    # save the prediction and target
    test_predictions.append(prediction)
    test_targets.append(target)

7.696385542168674
8.161676646706587
8.854782608695652
8.285106382978723
8.86
8.284357541899439
7.58
8.06268656716418
8.15079365079365
8.489876543209876
8.349350649350649
8.188135593220338
7.0227272727272725
8.193548387096774
7.86875
8.057575757575759
7.86875
7.927058823529409
8.319213973799126
8.658767772511847
8.383928571428573
6.620869565217391
8.15079365079365
8.16
8.288888888888888
8.116450216450215
8.16236559139785
7.053012048192772
7.997413793103448
8.488976377952756
8.658767772511847
8.932231404958676
8.244444444444445
8.436559139784947
8.191304347826087
8.15079365079365
8.303478260869564
8.223076923076922
7.96808510638298
8.317808219178083
9.313541666666666
8.211940298507463
8.22753623188406
8.287029288702929
8.284357541899439
8.465686274509803
8.35912408759124
8.488235294117647
8.51559633027523
6.813281249999999
7.849562682215742
7.8675000000000015
7.109767441860467
7.6185185185185205
6.620869565217391
6.065979381443299
8.340000000000002
8.285185185185185
8.539393939393939
6.6

In [220]:
def rmse(predictions, targets):
    assert len(predictions) == len(targets)
    predictions = np.array(predictions)
    targets = np.array(targets)
    squared_diffs = (predictions - targets) ** 2
    mse = np.mean(squared_diffs)
    rmse = np.sqrt(mse)
    return rmse
print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 3.799254154578746
test mse: 2.0861084593003807
